In [1]:
import mmh3

In [2]:
import functools

In [3]:
import iteration_utilities

In [4]:
import random

In [5]:
import numpy as np

In [6]:
import math

In [7]:
import tensorflow as tf

In [8]:
from tensorflow.python import debug as tf_debug

In [9]:
from enum import IntEnum

In [10]:
print("TensorFlow version: {}".format(tf.VERSION))

TensorFlow version: 1.8.0


In [11]:
# (x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()
training_data, testing_data = tf.keras.datasets.fashion_mnist.load_data()

In [12]:
number_of_classes = int(np.max(testing_data[1])) + 1
number_of_input_channels = 1

In [13]:
number_of_experts_per_class = 2

In [14]:
number_of_experts = number_of_experts_per_class * number_of_classes

In [15]:
number_of_hidden_layers = 16

In [16]:
number_of_features_per_layer = 2

In [17]:
number_of_computing_layers = number_of_hidden_layers + number_of_classes

In [18]:
number_of_layers = number_of_input_channels + number_of_computing_layers

In [19]:
preferred_number_samples_per_layer = number_of_layers # All the previous layers 

In [20]:
paddings = (0, 0), (2, 2), (2, 2)

In [21]:
random_seed = number_of_hidden_layers

In [22]:
weight_scale = 0.9

In [23]:
def offset_initializer():
    return tf.random_normal_initializer(stddev=3.0)
#     return tf.random_uniform_initializer(minval=-3.0, maxval=3.0)

In [24]:
def grid_sample(images, offset_x, offset_y):
    tf.assert_rank(images, 4)
    tf.assert_rank(offset_x, 4)
    tf.assert_rank(offset_y, 4)
    image_shape = tf.shape(images)
    current_batch_size = image_shape[0]
    width = image_shape[1]
    height = image_shape[2]
    number_of_features = image_shape[3]
    
    def check_width(tensor):
        if (tf.test.is_gpu_available()):
            return tensor
        else:
            return tf.maximum(tf.minimum(tensor, width - 1), 0)
    
    def check_height(tensor):
        if (tf.test.is_gpu_available()):
            return tensor
        else:
            return tf.maximum(tf.minimum(tensor, height - 1), 0)
    
    offset_left = tf.floor(offset_x)
    offset_top = tf.floor(offset_y)
    tf.assert_equal(tf.shape(offset_left), image_shape)
    tf.assert_equal(tf.shape(offset_top), image_shape)

    
    eps = 1e-7

    factor_right = offset_x - offset_left + eps
    factor_left = 1.0 + 2.0 * eps - factor_right
    factor_bottom = offset_y - offset_top + eps
    factor_top = 1.0 + 2.0 * eps - factor_bottom
    
    image_index, x_index, y_index, feature_index = tf.meshgrid(
        tf.range(current_batch_size, dtype=tf.int32),
        tf.range(width, dtype=tf.int32),
        tf.range(height, dtype=tf.int32),
        tf.range(number_of_features, dtype=tf.int32),
        indexing='ij',
    )

    left_index = check_width(x_index + tf.cast(offset_left, tf.int32))
    top_index = check_height(y_index + tf.cast(offset_top, tf.int32))

    right_index = left_index + 1
    bottom_index = top_index + 1
    
    tf.assert_rank(left_index, 4)
    tf.assert_rank(right_index, 4)
    tf.assert_rank(bottom_index, 4)
    tf.assert_rank(top_index, 4)
    
    images_top_left = tf.gather_nd(images, tf.stack((image_index, left_index, top_index, feature_index), axis=4))
    images_top_right = tf.gather_nd(images, tf.stack((image_index, right_index, top_index, feature_index), axis=4))
    images_bottom_left = tf.gather_nd(images, tf.stack((image_index, left_index, bottom_index, feature_index), axis=4))
    images_bottom_right = tf.gather_nd(images, tf.stack((image_index, right_index, bottom_index, feature_index), axis=4))

    tf.assert_rank(images_top_left, 4)
    tf.assert_rank(images_top_right, 4)
    tf.assert_rank(images_bottom_left, 4)
    tf.assert_rank(images_bottom_right, 4)

    lerp_top = factor_left * images_top_left + factor_right * images_top_right
    lerp_bottom = factor_left * images_bottom_left + factor_right * images_bottom_right
    output = factor_top * lerp_top + factor_bottom * lerp_bottom
    tf.assert_rank(output, 4)
    return output

In [25]:
def offnet(images, number_of_features, number_of_experts=None, name=None):
    with tf.variable_scope(name, default_name="offnet"):
        input_shape = images.shape
        if number_of_experts is None:
            number_of_experts = int(input_shape[0])
        number_of_input_features = int(input_shape[4])
        score_weight = tf.get_variable(
            name="score_weight",
            initializer=tf.random_normal_initializer(stddev=tf.sqrt(weight_scale / number_of_input_features)),
            dtype=tf.float32,
            shape=(number_of_experts, number_of_input_features, number_of_features)
        )
        offset_x_weight = tf.get_variable(
            name="offset_x_weight",
            initializer=tf.random_normal_initializer(stddev=tf.sqrt(weight_scale / number_of_input_features)),
            dtype=tf.float32,
            shape=(number_of_experts, number_of_input_features, number_of_features)
        )
        offset_y_weight = tf.get_variable(
            name="offset_y_weight",
            initializer=tf.random_normal_initializer(stddev=tf.sqrt(weight_scale / number_of_input_features)),
            dtype=tf.float32,
            shape=(number_of_experts, number_of_input_features, number_of_features)
        )
        offset_x_bias = tf.get_variable(
            name="offset_x_bias",
            initializer=offset_initializer(),
            dtype=tf.float32,
            shape=(number_of_experts, 1, number_of_features),
        )
        offset_y_bias = tf.get_variable(
            name="offset_y_bias",
            initializer=offset_initializer(),
            dtype=tf.float32,
            shape=(number_of_experts, 1,  number_of_features),
        )
#         def scale_initializer():
#             return tf.sqrt(weight_scale / number_of_input_features)
#         score_scale = tf.get_variable(
#             name="score_scale",
#             initializer=scale_initializer(),
#             dtype=tf.float32
#         )
#         offset_x_scale = tf.get_variable(
#             name="offset_x_scale",
#             initializer=scale_initializer(),
#             dtype=tf.float32
#         )
#         offset_y_scale = tf.get_variable(
#             name="offset_y_scale",
#             initializer=scale_initializer(),
#             dtype=tf.float32
#         )
        
        dynamic_image_shape = tf.shape(images)
        images_3d = tf.reshape(
            images,
            (
                number_of_experts,
                dynamic_image_shape[1] * dynamic_image_shape[2] * dynamic_image_shape[3],
                number_of_input_features
            )
        )
        def images_xw_plus_b(w, b):
            tf.matmul(images_3d, w) + b
        
        def to_4d(images):
            tf.assert_equal(tf.shape(images), (number_of_experts, dynamic_image_shape[1] * dynamic_image_shape[2] * dynamic_image_shape[3], number_of_features))
            return tf.reshape(
                images,
                (
                    number_of_experts * dynamic_image_shape[1],
                    dynamic_image_shape[2],
                    dynamic_image_shape[3],
                    number_of_features
                )
            )
        return tf.reshape(
            grid_sample(
                to_4d(tf.matmul(images_3d, score_weight)),
                to_4d(tf.matmul(images_3d, offset_x_weight) + offset_x_bias),
                to_4d(tf.matmul(images_3d, offset_y_weight) + offset_y_bias)
            ),
            (
                number_of_experts,
                dynamic_image_shape[1],
                dynamic_image_shape[2],
                dynamic_image_shape[3],
                number_of_features
            )
        )        

In [26]:
def model_fn(features, labels, mode, params, config):
    random.seed(number_of_hidden_layers)
    padded_input = tf.expand_dims(
        tf.expand_dims(
            tf.pad(tf.cast(features, tf.float32) / 255.0 - 0.5, paddings),
            axis=0
        ),
        axis=4
    ) + tf.constant(0.0, shape=(number_of_experts_per_class * number_of_classes, 1, 1, 1, 1))
    
    layers = functools.reduce(
        lambda layers, i: tf.concat(
            (
                layers,
                tf.layers.batch_normalization(
                    offnet(layers, number_of_features_per_layer, number_of_experts_per_class * number_of_classes),
                    training=mode is tf.estimator.ModeKeys.TRAIN
                )
            ),
            axis=4
        ),
        range(number_of_hidden_layers),
        padded_input
    )
    score_layers = offnet(layers, 1, number_of_experts_per_class * number_of_classes)
    dynamic_shape = tf.shape(layers)
    current_batch_size = dynamic_shape[1]
    scores = tf.transpose(
        tf.reduce_mean(
            tf.reshape(
                score_layers[:, :, 16, 16, 0],
                (number_of_classes, number_of_experts_per_class, current_batch_size)
            ),
            axis=(1)
        )
    )

    probabilities = tf.nn.softmax(logits=scores)
    predicted_classes = tf.argmax(scores, 1)
    predictions = {
        'probabilities' : probabilities,
        'scores': scores,
        'class': predicted_classes,
    }

    optimizer = tf.train.AdagradOptimizer(learning_rate=0.1)
    if labels is None:
        return tf.estimator.EstimatorSpec(
            mode=mode,
            predictions=predictions,
        )
    else:
        loss = tf.losses.softmax_cross_entropy(logits=scores, onehot_labels=tf.one_hot(labels, number_of_classes))
        eval_metric_ops = {
            'accuracy': tf.metrics.accuracy(labels=labels, predictions=predicted_classes)
        }
        update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
        with tf.control_dependencies(update_ops):
            return tf.estimator.EstimatorSpec(
                mode=mode,
                predictions=predictions,
                loss=loss,
                train_op=optimizer.minimize(loss, global_step=tf.train.get_global_step()),
                eval_metric_ops=eval_metric_ops,
            )

In [27]:
run_config = tf.estimator.RunConfig(
    model_dir=f"models/bnexpertoffnet{number_of_experts}x{number_of_features_per_layer}x{number_of_hidden_layers}",
    session_config=tf.ConfigProto(
        gpu_options=tf.GPUOptions(
            allow_growth=True,
        ),
        graph_options=tf.GraphOptions(
            optimizer_options=tf.OptimizerOptions(
                global_jit_level=tf.OptimizerOptions.ON_2,
                do_function_inlining=True,
                do_constant_folding=True,
                do_common_subexpression_elimination=True,
            ),
        )
    )
)

In [28]:
estimator = tf.estimator.Estimator(model_fn, config=run_config)

INFO:tensorflow:Using config: {'_model_dir': 'models/bnexpertoffnet20x2x16', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': gpu_options {
  allow_growth: true
}
graph_options {
  optimizer_options {
    do_common_subexpression_elimination: true
    do_constant_folding: true
    do_function_inlining: true
    global_jit_level: ON_2
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f5891239c18>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [29]:
batch_size = 32

In [30]:
def training_dataset():
    return tf.data.Dataset.from_tensor_slices(training_data).shuffle(1000).batch(batch_size)

In [31]:
def testing_dataset():
    return tf.data.Dataset.from_tensor_slices(testing_data).batch(batch_size)

In [32]:
# hook = tf_debug.TensorBoardDebugHook("localhost:6064")
# estimator.train(training_dataset,hooks=[hook])

In [ ]:
tf.estimator.train_and_evaluate(
    estimator,
    train_spec=tf.estimator.TrainSpec(training_dataset),
    eval_spec=tf.estimator.EvalSpec(testing_dataset, throttle_secs=1800)
)

INFO:tensorflow:Running training and evaluation locally (non-distributed).
INFO:tensorflow:Start train and evaluate loop. The evaluate will happen after 1800 secs (eval_spec.throttle_secs) or training is finished.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 1 into models/bnexpertoffnet20x2x16/model.ckpt.
INFO:tensorflow:loss = 2.4662516, step = 0
INFO:tensorflow:global_step/sec: 0.824902
INFO:tensorflow:loss = 0.48167908, step = 100 (121.228 sec)
INFO:tensorflow:global_step/sec: 0.862339
INFO:tensorflow:loss = 0.7118013, step = 200 (115.963 sec)
INFO:tensorflow:global_step/sec: 0.863082
INFO:tensorflow:loss = 0.45087776, step = 300 (115.864 sec)
INFO:tensorflow:global_step/sec: 0.861727
INFO:tensorflow:loss = 0.45472845, step = 400 (116.046 sec)
INFO:tensorf

INFO:tensorflow:global_step/sec: 0.862911
INFO:tensorflow:loss = 0.29908735, step = 4256 (115.887 sec)
INFO:tensorflow:global_step/sec: 0.862215
INFO:tensorflow:loss = 0.17814003, step = 4356 (115.980 sec)
INFO:tensorflow:global_step/sec: 0.862664
INFO:tensorflow:loss = 0.14851612, step = 4456 (115.921 sec)
INFO:tensorflow:global_step/sec: 0.862442
INFO:tensorflow:loss = 0.47812194, step = 4556 (115.949 sec)
INFO:tensorflow:Saving checkpoints for 4584 into models/bnexpertoffnet20x2x16/model.ckpt.
INFO:tensorflow:Loss for final step: 0.30497676.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-06-15-04:47:44
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from models/bnexpertoffnet20x2x16/model.ckpt-4584
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Evaluation [10/100]
INFO:tensorflow:Evaluation [20/100]
INFO:tensorflow:Evaluation [30/100]
IN

INFO:tensorflow:loss = 0.15852538, step = 7639
INFO:tensorflow:global_step/sec: 0.827365
INFO:tensorflow:loss = 0.29657665, step = 7739 (120.867 sec)
INFO:tensorflow:global_step/sec: 0.863321
INFO:tensorflow:loss = 0.12576973, step = 7839 (115.831 sec)
INFO:tensorflow:global_step/sec: 0.862603
INFO:tensorflow:loss = 0.2968474, step = 7939 (115.928 sec)
INFO:tensorflow:global_step/sec: 0.862207
INFO:tensorflow:loss = 0.20862076, step = 8039 (115.981 sec)
INFO:tensorflow:global_step/sec: 0.862399
INFO:tensorflow:loss = 0.3508477, step = 8139 (115.956 sec)
INFO:tensorflow:Saving checkpoints for 8153 into models/bnexpertoffnet20x2x16/model.ckpt.
INFO:tensorflow:global_step/sec: 0.856461
INFO:tensorflow:loss = 0.12365605, step = 8239 (116.760 sec)
INFO:tensorflow:global_step/sec: 0.861454
INFO:tensorflow:loss = 0.2637779, step = 8339 (116.083 sec)
INFO:tensorflow:global_step/sec: 0.862851
INFO:tensorflow:loss = 0.55057925, step = 8439 (115.895 sec)
INFO:tensorflow:global_step/sec: 0.864452


In [ ]:
# estimator.evaluate(input_fn=lambda:tf.data.Dataset.from_tensor_slices(testing_data).batch(3).take(1))

In [ ]:
# tuple(estimator.predict(input_fn=lambda:tf.data.Dataset.from_tensor_slices(testing_data).batch(3).take(1)))

In [ ]:
# estimator.train(input_fn=training_dataset, steps=1)